# Import Libraries

In [ ]:
#DataFrames manipulation
import pandas as pd

#Data download from Yahoo Finance
from yahooquery import Ticker

#Libraries for the Plotting
import holoviews as hv
from holoviews import opts, dim
from holoviews.plotting.links import RangeToolLink
from bokeh.models import HoverTool

#Librarie to save the plots to html object
import panel as pn
import param

hv.extension('bokeh')

# RSP vs. IVV - Performance

In [ ]:
#Download the Data from Yahoo Finance
tickers = Ticker('IVV RSP')
df = tickers.history(start = '2022-05-27')

#Reorganize the Dataframe
df = df.reset_index()
df = df.pivot(index='date', columns='symbol', values='close').reset_index()
df.columns = ['date', 'IVV', 'RSP']
df = df.rename(columns = {'date':'Date'})
df = df.set_index('Date')

#Calculate the cummultive Returns and then the indexed performance with base 100
df = df.pct_change() * 100
df = round(100 * (1 + df / 100).cumprod(), 2)
df = df.fillna(100)

#Add the relative performance of IVV and RSP
df['IVV_over_RSP'] = df['IVV'] / df['RSP'] * 100

In [ ]:
#Instantiate the Dataframe for the Graph
graph_df = df

#Generate all curves
def getCurves(n):
    for column in graph_df.columns:
        hover = HoverTool(tooltips=[("Date", "@Date{%F}"), (column, f"@{column+'{0,.00}'}")], formatters={'@Date': 'datetime'})  
        curve = hv.Curve(graph_df[column], label = column).opts(opts.Curve(tools=[hover]))
        curve = curve.opts(xticks=10)
        yield curve
        
source_curves, target_curves  = [], []
for curve in getCurves(2):
    
    src = curve.relabel('').opts(width=800, height=100, yaxis=None) 
    tgt = curve.opts(width=800, ylabel = 'Index (Base=100)')
    source_curves.append(src)
    target_curves.append(tgt)

# Link RangeTool for the first curves in the list.
RangeToolLink(source_curves[0],target_curves[0], axes=['x','y'])  

#Overlay the source and target curves 
overlaid_plot_src = hv.Overlay(source_curves).relabel('')
overlaid_plot_tgt = hv.Overlay(target_curves)

overlaid_plot_tgt = overlaid_plot_tgt.relabel('iShares Core S&P 500® ETF (IVV) vs. Invesco S&P 500® Equal Weight ETF (RSP)').opts(
    height=400, legend_position='top')

# Layout the plot
full_graph = (overlaid_plot_tgt + overlaid_plot_src).cols(1)
full_graph = full_graph.opts(merge_tools=False, shared_axes=False)

In [ ]:
#Save graph to Html
p = pn.panel(full_graph)
p.save('RSP_vs_IVV_Graph.html', embed = True)

# Current Top 10 Holdings of IVV and RSP

In [ ]:
#Download the data from iShares
url = 'https://www.ishares.com/us/products/239726/ishares-core-sp-500-etf/1467271812596.ajax?fileType=csv&fileName=IVV_holdings&dataType=fund'
curr_ivv_df = pd.read_csv(url, skiprows=9, delimiter=',', error_bad_lines=False)

#Limit the data for the Top 10 [will sum GOOGL with GOOG]
curr_top10_ivv_df = curr_ivv_df[:11]

#Keep only relevant columns
curr_top10_ivv_df = curr_top10_ivv_df[['Ticker', 'Name', 'Weight (%)']]

#Change the Ticker of Berkshire
curr_top10_ivv_df.loc[7, 'Ticker'] = 'BRK-B'

In [ ]:
#Download the data from iShares
url = 'https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker=RSP'
curr_rsp_df = pd.read_csv(url)

#Limit the columns of the DataFrame
curr_rsp_df = curr_rsp_df[['Holding Ticker', 'Weight']]

#Rename the columns
curr_rsp_df = curr_rsp_df.rename(columns = {'Holding Ticker': 'Ticker'})

#Round the Weight to 2 decimal cases
curr_rsp_df['Weight'] = round(curr_rsp_df['Weight'], 2)

#Take off the spaces from column Ticker
curr_rsp_df['Ticker'] = curr_rsp_df['Ticker'].str.replace(' ', '')

#Change the Ticker of Berkshire
curr_rsp_df['Ticker'] = curr_rsp_df['Ticker'].str.replace('BRK/B', 'BRK-B')

In [ ]:
#Merge both dataframes
curr_top10_complete = curr_top10_ivv_df.merge(curr_rsp_df, on = 'Ticker', how = 'left')

#Change the headers
curr_top10_complete = curr_top10_complete.rename(columns = {'Weight (%)': 'IVV', 'Weight': 'RSP'})

#Sum GOOGL and GOOG, drop GOOG, and change the name for 'Class' not to appear 
curr_top10_complete.loc[4] = curr_top10_complete.loc[4:5].sum()
curr_top10_complete = curr_top10_complete.drop(5)
curr_top10_complete.loc[4, 'Ticker'] = 'GOOGL'
curr_top10_complete.loc[4, 'Name'] = 'ALPHABET INC CLASS'

#Calculate the Top 10 Total Weight IVV
top10_weight_sum_ivv = curr_top10_complete['IVV'].sum()

#Calculate the Top 10 Total Weight IVV
top10_weight_sum_rsp = curr_top10_complete['RSP'].sum()

#Add the Data to the Curr_top10 dataframe
top10_weight_data = final_df = pd.DataFrame([{'Ticker':'-', 'Name':'Total', 'IVV':top10_weight_sum_ivv, 
                                              'RSP': top10_weight_sum_rsp}])
curr_top10_complete = pd.concat([curr_top10_complete, top10_weight_data], axis=0)

In [ ]:
#Create Holoviews table object for Top 10 Companies of RSP and IVV
curr_top10_complete_df_table = hv.Table(curr_top10_complete).opts(
    opts.Table(width=950, height=350, selectable = True, index_position = None, 
               title = 'Weight (%) in Current Top 10 Companies in the iShares Core S&P 500® ETF (IVV) vs. Invesco S&P 500® Equal Weight ETF (RSP)'))

In [ ]:
#Save the Plots
p = pn.panel(curr_top10_complete_df_table)
p.save('Current_IVV_RSP_Top10_Companies.html', embed = True)

# Historical Effective Number of Stocks in IVV

To calculate the Effective Number of Stocks, we need first to calculate the Herfindahl-Hirschman Index (HHI).

$$HHI = \sum_{k=1}^{n}w_{i}^{2}$$

Then the Effective Number of Stocks is:

$$Effective Number of Stocks = \frac{1}{HHI}$$

In [ ]:
#Current Effective Number of Stocks for RSP 
hhi_rsp_curr = ((curr_rsp_df['Weight']/100)**2).sum()
rsp_eff_stocks_curr = round(1/hhi_rsp_curr,0)

#Current Effective Number of Stocks for IVV 
hhi_ivv_curr = ((curr_ivv_df['Weight (%)']/100)**2).sum()
ivv_eff_stocks_curr = round(1/hhi_ivv_curr,0)

In [ ]:
#Instantiate an object with all the tabs in IVV_EDGAR_holdings_full file
ivv_edgar_holdings = pd.read_excel('IVV_EDGAR_holdings_full.xlsx', sheet_name=None)

In [ ]:
#Create an empty DataFrame where we will Concatenate the information
hist_ivv_eff_stocks_df = pd.DataFrame(columns=['Year', 'Effective Number of Stocks'])

#Run a Loop to get the Effective Number of Stocks for IVV  for each year
for year in ivv_edgar_holdings.keys():
    #Get the Effective Number of Stocks and Create a DataFrame with the data
    ivv_hhi_year = ((ivv_edgar_holdings[year]['Weight (%)'])**2).sum()
    ivv_eff_stocks_year = round(1/ivv_hhi_year,0)
    ivv_eff_stocks_year_df = pd.DataFrame([{'Year':year, 'Effective Number of Stocks':ivv_eff_stocks_year}])

    #Concatenate the New data in the hist_ivv_eff_stocks_df dataframe
    hist_ivv_eff_stocks_df = pd.concat([hist_ivv_eff_stocks_df, ivv_eff_stocks_year_df])

In [ ]:
#Convert the Year column to date showing the last day of the year
hist_ivv_eff_stocks_df['Year'] = pd.to_datetime(hist_ivv_eff_stocks_df['Year'], format='%Y') + pd.offsets.YearEnd()

#Get Current Date and Join current holdings to the Historical Effective Number of Stocks
current_date = pd.to_datetime((pd.Timestamp.today() - pd.offsets.BDay(1)).date())
current_ivv_eff_stocks_df = pd.DataFrame([{'Year':current_date, 'Effective Number of Stocks':ivv_eff_stocks_curr}])

#Join the Current Date data to the Historical data
hist_ivv_eff_stocks_df = pd.concat([hist_ivv_eff_stocks_df, current_ivv_eff_stocks_df])

#Rename Columns and set Index
hist_ivv_eff_stocks_df = hist_ivv_eff_stocks_df.rename(columns = {'Year':'Date', 'Effective Number of Stocks':'Effective_Number_Stocks'})
hist_ivv_eff_stocks_df = hist_ivv_eff_stocks_df.set_index('Date')

#Sort Values
hist_ivv_eff_stocks_df = hist_ivv_eff_stocks_df.sort_values(by='Date')

In [ ]:
#Instantiate the Dataframe for the Graph
graph_df = hist_ivv_eff_stocks_df

#Create the Graph
hover = HoverTool(tooltips=[("Date", "@Date{%F}"), ('Effective_Number_Stocks', f"@Effective_Number_Stocks")], formatters={'@Date': 'datetime'})  
hist_ivv_eff_stocks_graph = hv.Curve(graph_df).opts(opts.Curve(tools=[hover])).opts(width=800, height=400, ylabel = 'Effective Number of Stocks)',color = 'red', xticks=10
                                                               ).relabel('Historical Effective Number of Stocks of iShares Core S&P 500 ETF (IVV)')

In [ ]:
#Save the Plots
p = pn.panel(hist_ivv_eff_stocks_graph)
p.save('Historical_Effective_Number_Stocks.html', embed = True)